## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-02-20-50-00 +0000,wsj,Treasury Yields Have Mixed Weekly Performance ...,https://www.wsj.com/finance/u-s-treasury-yield...
1,2026-01-02-20-48-24 +0000,nyt,Saks C.E.O. Steps Down as Company Struggles to...,https://www.nytimes.com/2026/01/02/business/sa...
2,2026-01-02-20-48-00 +0000,wsj,Burger King Worker Charged With New Year’s Eve...,https://www.wsj.com/politics/national-security...
3,2026-01-02-20-35-08 +0000,nyt,Live Updates: Swiss Authorities Say Sparklers ...,https://www.nytimes.com/live/2026/01/02/world/...
4,2026-01-02-20-33-12 +0000,nypost,This Bronx nabe was the biggest whiner of 2025...,https://nypost.com/2026/01/02/us-news/wakefiel...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2384/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
18,new,35
19,year,31
30,fire,27
25,swiss,23
92,trump,20


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
166,2026-01-02-08-05-41 +0000,nypost,Swiss face painful task of identifying victims...,https://nypost.com/2026/01/02/world-news/swiss...,149
213,2026-01-01-22-55-18 +0000,wapo,"About 40 killed, 115 injured in New Year’s bar...",https://www.washingtonpost.com/world/2026/01/0...,149
83,2026-01-02-16-58-00 +0000,wsj,Champagne Sparklers Likely Ignited Ceiling in ...,https://www.wsj.com/world/europe/switzerland-s...,133
18,2026-01-02-19-55-55 +0000,nyt,What to Know About the Swiss Ski Resort Fire o...,https://www.nytimes.com/2026/01/01/world/europ...,132
19,2026-01-02-19-55-19 +0000,nypost,Survivors of Swiss New Year’s Eve fire broke w...,https://nypost.com/2026/01/02/world-news/survi...,132


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
166,149,2026-01-02-08-05-41 +0000,nypost,Swiss face painful task of identifying victims...,https://nypost.com/2026/01/02/world-news/swiss...
27,50,2026-01-02-19-30-41 +0000,nypost,Iran threatens American troops as ‘legitimate ...,https://nypost.com/2026/01/02/us-news/iran-thr...
180,40,2026-01-02-05-51-10 +0000,nypost,Kim Jong Un’s daughter visits family mausoleum...,https://nypost.com/2026/01/02/world-news/kim-j...
59,38,2026-01-02-17-39-02 +0000,nypost,Zelensky taps popular Ukraine spy chief to run...,https://nypost.com/2026/01/02/world-news/zelen...
33,35,2026-01-02-19-08-01 +0000,startribune,Tesla loses title as world's biggest electric ...,https://www.startribune.com/tesla-loses-title-...
135,34,2026-01-02-11-13-20 +0000,cbc,Sparkling flares in champagne bottles coming t...,https://www.cbc.ca/news/world/swiss-fire-bar-9...
134,30,2026-01-02-11-16-28 +0000,nypost,Young golfer Emanuele Galeppini named as first...,https://nypost.com/2026/01/02/world-news/golfe...
28,28,2026-01-02-19-28-23 +0000,cbc,Elon Musk's Grok AI says images of 'minors in ...,https://www.cbc.ca/news/world/xai-grok-safegua...
176,27,2026-01-02-06-12-12 +0000,nypost,Tommy Lee Jones joked he ‘fired’ daughter Vict...,https://nypost.com/2026/01/02/us-news/tommy-le...
184,26,2026-01-02-04-30-00 +0000,wsj,President Trump veered between business and pl...,https://www.wsj.com/politics/policy/golf-peace...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
